### Study of effect of national factors on home prices in US

**Task -** Using publically available data for the national factors that impact supply and demand of homes in US, build a model to study the effect of these variables on home prices.

**Approach -** The following variables are chosen for the study-
- Unemployment Rate
- Per Capita GDP
- Construction Prices
- Interest Rates
- Number of new houses supplied
- Percentage of population above 65
- Housing subsidies
- Number of Households

As a proxy to the home prices, S&P CASE-SHILLER Index is used. 

Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed and combined to create a datset. Data for different variables had different frequencies. So, to combine the data, necessary interpolations are made.

Linear Regression is used as most of the variables have high correlation with the target variable.

#### Importing Library

In [1]:
import pandas as pd
import numpy as np

#### Loading CSV File

In [2]:
df_cs = pd.read_csv("1 Home price.csv", names = ["DATE","Price"])
print(df_cs)

                 DATE      Price
0    observation_date  CSUSHPISA
1          2003-01-01     128.46
2          2003-02-01     129.36
3          2003-03-01     130.15
4          2003-04-01     130.88
..                ...        ...
245        2023-05-01     302.57
246        2023-06-01     304.59
247        2023-07-01     306.77
248        2023-08-01     309.16
249        2023-09-01     311.18

[250 rows x 2 columns]


In [3]:
# Remove the row with index 0
df_cs = df_cs.drop(0)

# Resetting index after dropping row 0
df_cs.reset_index(drop=True, inplace=True)

In [4]:
df_cs.dtypes

DATE     object
Price    object
dtype: object

In [5]:
#Changing dtype of date column
df_cs["DATE"] = pd.to_datetime(df_cs["DATE"])

In [6]:
df_cs['Price'] = pd.to_numeric(df_cs['Price'], errors='coerce')

In [7]:
df_cs.dtypes

DATE     datetime64[ns]
Price           float64
dtype: object

In [8]:
# Creating "Year" and "Month" columns
df_cs["Year"] = pd.DatetimeIndex(df_cs["DATE"]).year
df_cs["Month"] = pd.DatetimeIndex(df_cs["DATE"]).month
print(df_cs.shape)
print(df_cs)

(249, 4)
          DATE   Price  Year  Month
0   2003-01-01  128.46  2003      1
1   2003-02-01  129.36  2003      2
2   2003-03-01  130.15  2003      3
3   2003-04-01  130.88  2003      4
4   2003-05-01  131.74  2003      5
..         ...     ...   ...    ...
244 2023-05-01  302.57  2023      5
245 2023-06-01  304.59  2023      6
246 2023-07-01  306.77  2023      7
247 2023-08-01  309.16  2023      8
248 2023-09-01  311.18  2023      9

[249 rows x 4 columns]


In [9]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("3 Unemployment Rate.csv", names = ["DATE","Unemployment Rate"])
# Remove the row with index 0
df_unemp = df_unemp.drop(0)

# Resetting index after dropping row 0
df_unemp.reset_index(drop=True, inplace=True)

# Assuming df_unemp is your DataFrame
# Display the current indices of the DataFrame
print(df_unemp.index)

# Drop rows with indices 249 and 250 if they exist in the DataFrame
if all(idx in df_unemp.index for idx in [249, 250]):
    df_unemp.drop([249, 250], inplace=True)
    print("Rows with indices 249 and 250 dropped successfully.")
else:
    print("One or more indices not found in the DataFrame.")
print(df_unemp.shape)
df_unemp


RangeIndex(start=0, stop=251, step=1)
Rows with indices 249 and 250 dropped successfully.
(249, 2)


,DATE,Unemployment Rate
0,2003-01-01,5.8
1,2003-02-01,5.9
2,2003-03-01,5.9
3,2003-04-01,6.0
4,2003-05-01,6.1
...,...,...
244,2023-05-01,3.7
245,2023-06-01,3.6
246,2023-07-01,3.5
247,2023-08-01,3.8


In [10]:
df_unemp.dtypes

DATE                 object
Unemployment Rate    object
dtype: object

In [11]:
#Changing dtype of date column
df_unemp["DATE"] = pd.to_datetime(df_unemp["DATE"])
df_unemp['Unemployment Rate'] = pd.to_numeric(df_unemp['Unemployment Rate'], errors='coerce')

In [12]:
df_unemp.dtypes

DATE                 datetime64[ns]
Unemployment Rate           float64
dtype: object

In [13]:
# Reading Unemployment Rate Data into a dataframe
df_income = pd.read_csv("4 Income.csv", names = ["DATE","Income"])
# Remove the row with index 0
df_income = df_income.drop(0)

# Resetting index after dropping row 0
df_income.reset_index(drop=True, inplace=True)

# Drop rows with indices 249 if they exist in the DataFrame
if all(idx in df_income.index for idx in [249]):
    df_income.drop([249], inplace=True)
    print("Rows with indices 249  dropped successfully.")
else:
    print("One or more indices not found in the DataFrame.")

print(df_income.shape)
df_income

Rows with indices 249  dropped successfully.
(249, 2)


,DATE,Income
0,2003-01-01,10710.4
1,2003-02-01,10674.0
2,2003-03-01,10696.5
3,2003-04-01,10752.7
4,2003-05-01,10832.0
...,...,...
244,2023-05-01,16818.5
245,2023-06-01,16809.5
246,2023-07-01,16796.9
247,2023-08-01,16799.7


In [14]:
#Changing dtype of date column
df_income["DATE"] = pd.to_datetime(df_income["DATE"])
df_income['Income'] = pd.to_numeric(df_income['Income'], errors='coerce')

In [15]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("2 Intrested Rate.csv", names = ["DATE","Interest Rate"])
# Remove the row with index 0
df_Fed_rate = df_Fed_rate.drop(0)

# Resetting index after dropping row 0
df_Fed_rate.reset_index(drop=True, inplace=True)

# Assuming df_unemp is your DataFrame
# Display the current indices of the DataFrame
print(df_Fed_rate.index)

# Drop rows with indices 249 and 250 if they exist in the DataFrame
if all(idx in df_Fed_rate.index for idx in [249, 250]):
    df_Fed_rate.drop([249, 250], inplace=True)
    print("Rows with indices 249 and 250 dropped successfully.")
else:
    print("One or more indices not found in the DataFrame.")
    
print(df_Fed_rate.shape)
df_Fed_rate

RangeIndex(start=0, stop=251, step=1)
Rows with indices 249 and 250 dropped successfully.
(249, 2)


,DATE,Interest Rate
0,2003-01-01,1.24
1,2003-02-01,1.26
2,2003-03-01,1.25
3,2003-04-01,1.26
4,2003-05-01,1.26
...,...,...
244,2023-05-01,5.06
245,2023-06-01,5.08
246,2023-07-01,5.12
247,2023-08-01,5.33


In [16]:
#Changing dtype of date column
df_Fed_rate["DATE"] = pd.to_datetime(df_Fed_rate["DATE"])
df_Fed_rate['Interest Rate'] = pd.to_numeric(df_Fed_rate['Interest Rate'], errors='coerce')

In [17]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("5 Per Capital GDP.csv", names = ["DATE", "Per_Capital_GDP"])
# Remove the row with index 0
df_pcgdp = df_pcgdp.drop(0)

# Resetting index after dropping row 0
df_pcgdp.reset_index(drop=True, inplace=True)
print(df_pcgdp.shape)
df_pcgdp

(83, 2)


,DATE,Per_Capital_GDP
0,2003-01-01,50462
1,2003-04-01,50796
2,2003-07-01,51512
3,2003-10-01,51986
4,2004-01-01,52179
...,...,...
78,2022-07-01,65462
79,2022-10-01,65783
80,2023-01-01,66078
81,2023-04-01,66341


The data is quarterly. We will impute for other months using linear interpolation after we create the final dataframe combining all the data.

In [18]:
#Changing dtype of date column
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_pcgdp['Per_Capital_GDP'] = pd.to_numeric(df_pcgdp['Per_Capital_GDP'], errors='coerce')

In [19]:
df_pcgdp.dtypes

DATE               datetime64[ns]
Per_Capital_GDP             int64
dtype: object

In [20]:
# Reading Construction Price Index Data into a dataframe
df_cons_price_index = pd.read_csv("7 Construction Price Index.csv", names = ["DATE", "Cons_Materials_Price"])
# Remove the row with index 0
df_cons_price_index= df_cons_price_index.drop(0)

# Resetting index after dropping row 0
df_cons_price_index.reset_index(drop=True, inplace=True)

# Remove the row with index 249
df_cons_price_index= df_cons_price_index.drop(249)

# Resetting index after dropping row 249
df_cons_price_index.reset_index(drop=True, inplace=True)

print(df_cons_price_index)

           DATE Cons_Materials_Price
0    2003-01-01              144.400
1    2003-02-01              145.200
2    2003-03-01              145.200
3    2003-04-01              145.900
4    2003-05-01              145.800
..          ...                  ...
244  2023-05-01              337.473
245  2023-06-01              337.336
246  2023-07-01              334.576
247  2023-08-01              333.980
248  2023-09-01              332.224

[249 rows x 2 columns]


In [21]:
#Changing dtype of date column
df_cons_price_index["DATE"] = pd.to_datetime(df_cons_price_index["DATE"])
df_cons_price_index['Cons_Materials_Price'] = pd.to_numeric(df_cons_price_index['Cons_Materials_Price'], errors='coerce')

In [22]:
df_cons_price_index.dtypes

DATE                    datetime64[ns]
Cons_Materials_Price           float64
dtype: object

In [23]:
# Reading Per New Construction MAterial into a dataframe
df_cons_mtl = pd.read_csv("6 New Constructed Units.csv", names = ["DATE", "Cons_Materials"])
# Remove the row with index 0
df_cons_mtl= df_cons_mtl.drop(0)

# Resetting index after dropping row 0
df_cons_mtl.reset_index(drop=True, inplace=True)

# Remove the row with index 249
df_cons_mtl= df_cons_mtl.drop(249)

# Resetting index after dropping row 249
df_cons_mtl.reset_index(drop=True, inplace=True)

print(df_cons_mtl)

           DATE Cons_Materials
0    2003-01-01           1654
1    2003-02-01           1688
2    2003-03-01           1638
3    2003-04-01           1662
4    2003-05-01           1733
..          ...            ...
244  2023-05-01           1534
245  2023-06-01           1492
246  2023-07-01           1334
247  2023-08-01           1370
248  2023-09-01           1478

[249 rows x 2 columns]


In [24]:
#Changing dtype of date column
df_cons_mtl["DATE"] = pd.to_datetime(df_cons_mtl["DATE"])
df_cons_mtl['Cons_Materials'] = pd.to_numeric(df_cons_mtl['Cons_Materials'], errors='coerce')

In [25]:
df_cons_mtl.dtypes

DATE              datetime64[ns]
Cons_Materials             int64
dtype: object

In [26]:
# Housing Subsidies

df_subsidy = pd.read_csv("11 Housing Subsidies.csv", names = ["DATE", "Subsidy"])
# Remove the row with index 0
df_subsidy= df_subsidy.drop(0)

# Resetting index after dropping row 0
df_subsidy.reset_index(drop=True, inplace=True)

print(df_subsidy.shape)
df_subsidy

(20, 2)


,DATE,Subsidy
0,2003-01-01,25.930
1,2004-01-01,27.201
2,2005-01-01,27.651
3,2006-01-01,28.604
4,2007-01-01,29.512
5,2008-01-01,29.876
6,2009-01-01,32.883
7,2010-01-01,32.669
8,2011-01-01,34.230
9,2012-01-01,33.283


In [27]:
#Changing dtype of date column
df_subsidy["DATE"] = pd.to_datetime(df_subsidy["DATE"])
df_subsidy['Subsidy'] = pd.to_numeric(df_subsidy['Subsidy'], errors='coerce')

In [28]:
df_subsidy.dtypes

DATE       datetime64[ns]
Subsidy           float64
dtype: object

In [29]:
# Number of households

df_households = pd.read_csv("12 Total House Hold.csv", names = ["DATE", "Num_Households"])
# Remove the row with index 0
df_households= df_households.drop(0)

# Resetting index after dropping row 0
df_households.reset_index(drop=True, inplace=True)

# Remove the row with index 20
df_households= df_households.drop(20)

# Resetting index after dropping row 20
df_households.reset_index(drop=True, inplace=True)

print(df_households)

          DATE Num_Households
0   2003-01-01         111278
1   2004-01-01         112000
2   2005-01-01         113343
3   2006-01-01         114384
4   2007-01-01         116011
5   2008-01-01         116783
6   2009-01-01         117181
7   2010-01-01         117538
8   2011-01-01         119927
9   2012-01-01         121084
10  2013-01-01         122459
11  2014-01-01         123229
12  2015-01-01         124587
13  2016-01-01         125819
14  2017-01-01         126224
15  2018-01-01         127586
16  2019-01-01         128579
17  2020-01-01         128451
18  2021-01-01         129224
19  2022-01-01         131202


In [30]:
#Changing dtype of date column
df_households["DATE"] = pd.to_datetime(df_households["DATE"])
df_households['Num_Households'] = pd.to_numeric(df_households['Num_Households'], errors='coerce')

In [31]:
df_households.dtypes

DATE              datetime64[ns]
Num_Households             int64
dtype: object

In [32]:
df_pcgdp

,DATE,Per_Capital_GDP
0,2003-01-01,50462
1,2003-04-01,50796
2,2003-07-01,51512
3,2003-10-01,51986
4,2004-01-01,52179
...,...,...
78,2022-07-01,65462
79,2022-10-01,65783
80,2023-01-01,66078
81,2023-04-01,66341


In [33]:
# Merging Per Capita GDP (Quarterly data)
df_pcgdp["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df = pd.merge(df_cs,df_pcgdp, how = "left")
df

,DATE,Price,Year,Month,Per_Capital_GDP
0,2003-01-01,128.46,2003,1,50462.0
1,2003-02-01,129.36,2003,2,NaN
2,2003-03-01,130.15,2003,3,NaN
3,2003-04-01,130.88,2003,4,50796.0
4,2003-05-01,131.74,2003,5,NaN
...,...,...,...,...,...
244,2023-05-01,302.57,2023,5,NaN
245,2023-06-01,304.59,2023,6,NaN
246,2023-07-01,306.77,2023,7,67083.0
247,2023-08-01,309.16,2023,8,NaN


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 248
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DATE             249 non-null    datetime64[ns]
 1   Price            249 non-null    float64       
 2   Year             249 non-null    int64         
 3   Month            249 non-null    int64         
 4   Per_Capital_GDP  83 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 11.7 KB


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 248
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   DATE             249 non-null    datetime64[ns]
 1   Price            249 non-null    float64       
 2   Year             249 non-null    int64         
 3   Month            249 non-null    int64         
 4   Per_Capital_GDP  83 non-null     float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 11.7 KB


In [36]:
df_unemp.dtypes

DATE                 datetime64[ns]
Unemployment Rate           float64
dtype: object

In [37]:
# Concating dataframes having monthly data to create one dataframe
df = pd.DataFrame()
df_bymonth = [df_cs, df_unemp, df_cons_price_index, df_Fed_rate,df_pcgdp,df_households,df_income,df_subsidy]
for df1 in df_bymonth:
    df1 = df1.set_index("DATE")
    df = pd.concat([df,df1], axis = 1)
print(df.shape)
df.head()

(249, 10)


,Price,Year,Month,Unemployment Rate,Cons_Materials_Price,Interest Rate,Per_Capital_GDP,Num_Households,Income,Subsidy
DATE,,,,,,,,,,
2003-01-01,128.46,2003,1,5.8,144.4,1.24,50462.0,111278.0,10710.4,25.93
2003-02-01,129.36,2003,2,5.9,145.2,1.26,NaN,NaN,10674.0,NaN
2003-03-01,130.15,2003,3,5.9,145.2,1.25,NaN,NaN,10696.5,NaN
2003-04-01,130.88,2003,4,6.0,145.9,1.26,50796.0,NaN,10752.7,NaN
2003-05-01,131.74,2003,5,6.1,145.8,1.26,NaN,NaN,10832.0,NaN


In [38]:
df_households.dtypes

DATE              datetime64[ns]
Num_Households             int64
dtype: object

In [39]:
df_subsidy.dtypes

DATE       datetime64[ns]
Subsidy           float64
dtype: object

In [40]:
df_income.dtypes

DATE      datetime64[ns]
Income           float64
dtype: object

In [41]:
df.head()

,Price,Year,Month,Unemployment Rate,Cons_Materials_Price,Interest Rate,Per_Capital_GDP,Num_Households,Income,Subsidy
DATE,,,,,,,,,,
2003-01-01,128.46,2003,1,5.8,144.4,1.24,50462.0,111278.0,10710.4,25.93
2003-02-01,129.36,2003,2,5.9,145.2,1.26,NaN,NaN,10674.0,NaN
2003-03-01,130.15,2003,3,5.9,145.2,1.25,NaN,NaN,10696.5,NaN
2003-04-01,130.88,2003,4,6.0,145.9,1.26,50796.0,NaN,10752.7,NaN
2003-05-01,131.74,2003,5,6.1,145.8,1.26,NaN,NaN,10832.0,NaN


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 249 entries, 2003-01-01 to 2023-09-01
Freq: MS
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Price                 249 non-null    float64
 1   Year                  249 non-null    int64  
 2   Month                 249 non-null    int64  
 3   Unemployment Rate     249 non-null    float64
 4   Cons_Materials_Price  249 non-null    float64
 5   Interest Rate         249 non-null    float64
 6   Per_Capital_GDP       83 non-null     float64
 7   Num_Households        20 non-null     float64
 8   Income                249 non-null    float64
 9   Subsidy               20 non-null     float64
dtypes: float64(8), int64(2)
memory usage: 21.4 KB


In [43]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capital_GDP"] = df["Per_Capital_GDP"].interpolate()

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 249 entries, 2003-01-01 to 2023-09-01
Freq: MS
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Price                 249 non-null    float64
 1   Year                  249 non-null    int64  
 2   Month                 249 non-null    int64  
 3   Unemployment Rate     249 non-null    float64
 4   Cons_Materials_Price  249 non-null    float64
 5   Interest Rate         249 non-null    float64
 6   Per_Capital_GDP       249 non-null    float64
 7   Num_Households        20 non-null     float64
 8   Income                249 non-null    float64
 9   Subsidy               20 non-null     float64
dtypes: float64(8), int64(2)
memory usage: 21.4 KB


In [45]:
# Assuming df is your DataFrame
# Fill NaN values in 'Num_Households' and 'Subsidy' columns with their respective medians
median_num_households = df['Num_Households'].median()
median_subsidy = df['Subsidy'].median()

df['Num_Households'].fillna(median_num_households, inplace=True)
df['Subsidy'].fillna(median_subsidy, inplace=True)

In [46]:
df.isna().sum()

Price                   0
Year                    0
Month                   0
Unemployment Rate       0
Cons_Materials_Price    0
Interest Rate           0
Per_Capital_GDP         0
Num_Households          0
Income                  0
Subsidy                 0
dtype: int64

In [47]:
df.shape

(249, 10)

In [48]:
df.tail()

,Price,Year,Month,Unemployment Rate,Cons_Materials_Price,Interest Rate,Per_Capital_GDP,Num_Households,Income,Subsidy
DATE,,,,,,,,,,
2023-05-01,302.57,2023,5,3.7,337.473,5.06,66588.333333,121771.5,16818.5,33.5445
2023-06-01,304.59,2023,6,3.6,337.336,5.08,66835.666667,121771.5,16809.5,33.5445
2023-07-01,306.77,2023,7,3.5,334.576,5.12,67083.000000,121771.5,16796.9,33.5445
2023-08-01,309.16,2023,8,3.8,333.980,5.33,67083.000000,121771.5,16799.7,33.5445
2023-09-01,311.18,2023,9,3.8,332.224,5.33,67083.000000,121771.5,16804.8,33.5445


This is our preprocessed datset. Let's save it as "prepared_dataset.csv".

In [49]:
df.to_csv("prepared_dataset1.csv")

In [50]:
df = pd.read_csv("prepared_dataset1.csv").set_index("DATE")
df.head()

,Price,Year,Month,Unemployment Rate,Cons_Materials_Price,Interest Rate,Per_Capital_GDP,Num_Households,Income,Subsidy
DATE,,,,,,,,,,
2003-01-01,128.46,2003,1,5.8,144.4,1.24,50462.000000,111278.0,10710.4,25.9300
2003-02-01,129.36,2003,2,5.9,145.2,1.26,50573.333333,121771.5,10674.0,33.5445
2003-03-01,130.15,2003,3,5.9,145.2,1.25,50684.666667,121771.5,10696.5,33.5445
2003-04-01,130.88,2003,4,6.0,145.9,1.26,50796.000000,121771.5,10752.7,33.5445
2003-05-01,131.74,2003,5,6.1,145.8,1.26,51034.666667,121771.5,10832.0,33.5445
